In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import torch
import re
from sklearn.metrics import balanced_accuracy_score, roc_auc_score,accuracy_score,precision_recall_fscore_support
from Constants import *
from Preprocessing import *
from Models import *
import copy
from Utils import *
from DeepSurvivalModels import *
pd.set_option('display.max_rows', 200)



In [3]:
data = DTDataset(use_smote=False)
data.processed_df.T
data.get_input_state(1).shape
# data.processed_df#.shape, len(data.processed_df.index.unique())

(536, 60)

In [6]:
data.processed_df
dt_ids = data.processed_df.index.values
dt_ids

array([    3,     5,     6,     7,     8,     9,    10,    11,    13,
          14,    15,    16,    17,    18,    21,    23,    24,    25,
          26,    27,    28,    31,    32,    33,    35,    36,    37,
          38,    39,    40,    41,    42,    44,    45,    47,    48,
          49,    50,    51,    53,    55,    56,    57,    60,    64,
          65,    67,    68,    69,    71,    74,    75,    77,    78,
          79,    80,    81,    82,    87,    88,    91,    94,    96,
          99,   103,   109,   116,   117,   119,   120,   121,   125,
         133,   148,   150,   153,   168,   178,   181,   183,   184,
         185,   186,   187,   188,   189,   190,   191,   192,   193,
         194,   195,   196,   197,   198,   199,   200,   201,   202,
         203,   204,   205,   206,   207,   208,   209,   210,   211,
         212,   213,   214,   215,   216,   217,   218,   219,   220,
         221,   222,   223,   224,   225,   226,   227,   228,   229,
         230,   231,

In [7]:
train_ids, test_ids = get_tt_split()

In [8]:
def overlap(v1,v2):
    return sorted([vv for vv in v1 if vv in v2])

def get_id_mapped_r01(file = '../data/key_map.xlsx',reverse=False):
    df = pd.read_excel('../data/key_map.xlsx').drop('Unnamed: 0',axis=1)
    df['mdasi_id'] = df['STIEFEL'].apply(lambda x: int(x.replace("STIEFEL_",'')))
    df = df[['mdasi_id','ID']]
    if reverse:
        return df.set_index('mdasi_id').to_dict()['ID']
    idmap = df.set_index('ID').to_dict()['mdasi_id']
    r01 = pd.read_csv('../data/distance_csv.csv').drop('Unnamed: 0',axis=1)
    r01['old_id'] = r01.id
    r01['id'] = r01['id'].apply(lambda x: idmap.get(x,x))
    return r01

camprt = pd.read_csv('../data/camprtdists.csv').rename({'ID':'id'},axis=1)
mdasi = pd.read_csv('../data/MDASI_0909201_surgery_updated.csv')
r01 = get_id_mapped_r01()
camprt_overlap = set(camprt.id.values.astype(int)).intersection(data.processed_df.index.values.astype(int))
r01_overlap = set(r01.id.values.astype(int)).intersection(data.processed_df.index.values.astype(int))
mdasi_overlap = set(mdasi.id.values.astype(int)).intersection(data.processed_df.index.values.astype(int))
len(camprt_overlap), len(r01_overlap), len(camprt_overlap.union(r01_overlap)),len(mdasi_overlap), len(mdasi_overlap.intersection(r01_overlap.union(camprt_overlap)))

(165, 77, 215, 185, 127)

In [9]:

clusters = pd.read_excel('../data/mdasi_dose_clusters_v2.xlsx')
cluster_overlap = overlap(clusters.id.values.astype('int'),dt_ids)
len(cluster_overlap), len(overlap(clusters.id.values.astype('int'),train_ids)), len(overlap(clusters.id.values.astype('int'),test_ids))

(144, 102, 42)

In [10]:
train_cluster_ids = overlap(clusters.id.values.astype('int'),train_ids)
test_cluster_ids = overlap(clusters.id.values.astype('int'),test_ids)

In [11]:
xctrain = data.get_input_state(step=1,ids=train_cluster_ids).drop(Const.decisions[0],axis=1)
xctest = data.get_input_state(step=1,ids=test_cluster_ids).drop(Const.decisions[0],axis=1)
yctrain = clusters.set_index('id').loc[train_cluster_ids]
yctest = clusters.set_index('id').loc[test_cluster_ids]
cluster_cols = [c for c in yctrain.columns if '_filtered' in c]
yctrain = yctrain[cluster_cols]
yctest = yctest[cluster_cols]
yctest.shape, yctrain.shape, cluster_cols

((42, 7),
 (102, 7),
 ['dose_clusters_drymouth_filtered',
  'dose_clusters_voice_filtered',
  'dose_clusters_choke_filtered',
  'dose_clusters_swallow_filtered',
  'dose_clusters_mucus_filtered',
  'dose_clusters_mucositis_filtered',
  'dose_clusters_taste_filtered'])

In [12]:
yctest

,dose_clusters_drymouth_filtered,dose_clusters_voice_filtered,dose_clusters_choke_filtered,dose_clusters_swallow_filtered,dose_clusters_mucus_filtered,dose_clusters_mucositis_filtered,dose_clusters_taste_filtered
id,,,,,,,
7,2,0,1,2,0,0,0
9,1,1,2,1,1,2,1
10,2,2,0,0,0,0,0
25,2,0,0,2,0,1,0
31,1,1,2,1,1,2,2
35,0,1,2,1,2,2,1
44,2,0,0,2,1,1,1
45,1,1,2,2,1,0,2
47,1,0,0,2,1,1,1


In [13]:
from Models import ClusterImputer
imputer = ClusterImputer(xctrain.shape[1],yctest.shape[1])
imputer

ClusterImputer(
  (input_dropout): Dropout(p=0, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=59, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU()
  )
  (batchnorm): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (relu): Softplus(beta=1, threshold=20)
  (sigmoid): Sigmoid()
  (softmax): Softmax(dim=1)
  (final_layers): ModuleList(
    (0-6): 7 x Linear(in_features=100, out_features=3, bias=True)
  )
)

In [14]:
xtrain = df_to_torch(xctrain)
ytrain = df_to_torch(yctrain)
xtest = df_to_torch(xctest)
ytest = df_to_torch(yctest)
imputer.fit_normalizer(xtrain)
imputer(xtrain)

[tensor([[0.3313, 0.3413, 0.3274],
         [0.3306, 0.3568, 0.3126],
         [0.3713, 0.3253, 0.3035],
         [0.3472, 0.3290, 0.3238],
         [0.3325, 0.3542, 0.3133],
         [0.3613, 0.3145, 0.3242],
         [0.3515, 0.3270, 0.3214],
         [0.3572, 0.3293, 0.3135],
         [0.3459, 0.3302, 0.3240],
         [0.3339, 0.3579, 0.3082],
         [0.3183, 0.3710, 0.3107],
         [0.2387, 0.3633, 0.3980],
         [0.3390, 0.3535, 0.3074],
         [0.3666, 0.3357, 0.2977],
         [0.3482, 0.3334, 0.3184],
         [0.3049, 0.3570, 0.3381],
         [0.3094, 0.4000, 0.2905],
         [0.2951, 0.3728, 0.3321],
         [0.3405, 0.3537, 0.3058],
         [0.3159, 0.3808, 0.3033],
         [0.3318, 0.3410, 0.3272],
         [0.3512, 0.3057, 0.3431],
         [0.3268, 0.3340, 0.3392],
         [0.3460, 0.3471, 0.3070],
         [0.3227, 0.3480, 0.3293],
         [0.2938, 0.3616, 0.3446],
         [0.3363, 0.3403, 0.3234],
         [0.3722, 0.3526, 0.2752],
         [0.3180, 0.

In [46]:
def cluster_loss(ytrue,ypred,weights=None):
    if weights is None:
        weights = [1 for i in range(ytrue.shape[1])]
    loss = 0
    nloss = torch.nn.BCELoss()
    for i in range(len(weights)):
        iloss = nloss(ypred[:,i],ytrue[:,i])*weights[i]
        loss += iloss
    return loss

epochs = 100
best_val_loss = 10000
best_loss_metrics = {}
patience = 10
save_file = '../data/models/clusterModel.tar'
optimizer = torch.optim.Adam(imputer.parameters(),lr=.01)
lossF = torch.nn.CrossEntropyLoss()
for epoch in range(epochs):
    ypreds = imputer(xtrain)
    losses = 0
    for i,ypred in enumerate(ypreds):
        losses+= lossF(ypred,ytrain[:,i].long())
    losses.backward()
    optimizer.step()
    if loss < 

tensor(8.2033, grad_fn=<AddBackward0>)
tensor(8.1640, grad_fn=<AddBackward0>)
tensor(8.0954, grad_fn=<AddBackward0>)
tensor(8.1542, grad_fn=<AddBackward0>)
tensor(8.1836, grad_fn=<AddBackward0>)
tensor(8.1346, grad_fn=<AddBackward0>)
tensor(8.1836, grad_fn=<AddBackward0>)
tensor(8.1444, grad_fn=<AddBackward0>)
tensor(8.0660, grad_fn=<AddBackward0>)
tensor(8.1346, grad_fn=<AddBackward0>)
tensor(8.1444, grad_fn=<AddBackward0>)
tensor(8.1640, grad_fn=<AddBackward0>)
tensor(8.1934, grad_fn=<AddBackward0>)
tensor(8.1248, grad_fn=<AddBackward0>)
tensor(8.1640, grad_fn=<AddBackward0>)
tensor(8.0954, grad_fn=<AddBackward0>)
tensor(8.1444, grad_fn=<AddBackward0>)
tensor(8.1640, grad_fn=<AddBackward0>)
tensor(8.2621, grad_fn=<AddBackward0>)
tensor(8.2033, grad_fn=<AddBackward0>)
tensor(8.0660, grad_fn=<AddBackward0>)
tensor(8.1640, grad_fn=<AddBackward0>)
tensor(8.2229, grad_fn=<AddBackward0>)
tensor(8.1444, grad_fn=<AddBackward0>)
tensor(8.0954, grad_fn=<AddBackward0>)
tensor(8.2033, grad_fn=<A

In [90]:
def get_unique_sequence(array):
    #converts a row of boolean values to a unique number e.g. [1,1,0] => 11, [0,0,1] => 100
    uniqueify = lambda r: torch.sum(torch.stack([i*(10**ii) for ii,i in enumerate(r)]))
    return torch_apply_along_axis(uniqueify,array)

def train_decision_model_triplet(
    tmodel1,
    tmodel2,
    tmodel3,
    smodel3,
    use_default_split=True,
    use_bagging_split=False,
    use_attention=True,
    lr=.001,
    epochs=10000,
    patience=5,
    weights=[0,.5,.5,0], #realtive weight of survival, feeding tube, aspiration, andl lrc
    tweights=[1,1,1,0], #weight for OS, LRC, FDM, and event (any + FT or AS at 6m) as time to event in weeks
    opt_weights=[1,1,1], #weights for policy model for optimal decisions
    imitation_weights=[.5,1,1],#weights of imitation decisions, because ic overtrains too quickly
    imitation_weight=1,
    reward_weight=1,
    imitation_triplet_weight=2,
    reward_triplet_weight = 2,
    shufflecol_chance = 0.2,
    split=.7,
    resample_training=False,
    save_path='../data/models/',
    file_suffix='',
    verbose=True,
    use_gpu=False,
    **model_kwargs,
):
    tmodel1.eval()
    tmodel2.eval()
    tmodel3.eval()

    train_ids, test_ids = get_tt_split(use_default_split=use_default_split,use_bagging_split=use_bagging_split,resample_training=resample_training)
    true_ids = train_ids + test_ids #for saving memory without upsampling

    dataset = DTDataset()
    data = dataset.processed_df.copy()
    
    def get_dlt(state):
        if state == 2:
            return data[Const.dlt2].copy()
        d = data[Const.dlt1].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_pd(state):
        if state == 2:
            return data[Const.primary_disease_states2].copy()
        d = data[Const.primary_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_nd(state):
        if state == 2:
            return data[Const.nodal_disease_states2].copy()
        d = data[Const.nodal_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_cc(state):
        res = data[Const.ccs].copy()
        if state == 1:
            res.values[:,:] = np.zeros(res.values.shape)
        return res
    
    def get_mod(state):
        res = data[Const.modifications].copy()
        #this should have an ic condition but we don't use it anumore anywa
        return res
        
    outcomedf = data[Const.outcomes]
    baseline = dataset.get_state('baseline')
    
    def formatdf(d,dids=train_ids):
        d = df_to_torch(d.loc[dids]).to(model.get_device())
        return d
    
    def makegrad(v):
        if not v.requires_grad:
            v.requires_grad=True
        return v
    
    if use_attention:
        model = DecisionAttentionModel(baseline.shape[1],**model_kwargs)
    else:
        model_kwargs = {k:v for k,v in model_kwargs.items() if 'attention' not in k and 'embed' not in k}
        model = DecisionModel(baseline.shape[1],**model_kwargs)
        
    device = 'cpu'
    if use_gpu and torch.cuda.is_available():
        device = 'cuda'
        
    model.set_device(device)

    tmodel1.set_device(device)
    tmodel2.set_device(device)
    tmodel3.set_device(device)
    smodel3.set_device(device)
    hashcode = str(hash(','.join([str(i) for i in train_ids])))
    
    save_file = save_path + 'model_' + model.identifier +'_hash' + hashcode + file_suffix + '.tar'
    model.fit_normalizer(df_to_torch(baseline.loc[train_ids]))
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    
    
    optimal_train,transitions_train = calc_optimal_decisions(dataset,train_ids,tmodel1,tmodel2,tmodel3,smodel3,
                                           weights=weights,tweights=tweights,
                                          )
    optimal_test,transitions_test = calc_optimal_decisions(dataset,test_ids,tmodel1,tmodel2,tmodel3,smodel3,
                                          weights=weights,tweights=tweights,
                                         )
    optimal_train = optimal_train.to(model.get_device())
    optimal_test = optimal_test.to(model.get_device())
    mse = torch.nn.MSELoss()
    bce = torch.nn.BCELoss()
    
    def compare_decisions(d1,d2,d3,ids):
#         ypred = np.concatenate([dd.cpu().detach().numpy().reshape(-1,1) for dd in [d1,d2,d3]],axis=1)
        ytrue = df_to_torch(outcomedf.loc[ids])
        dloss = bce(d1.view(-1),ytrue[:,0])
        dloss += bce(d2.view(-1),ytrue[:,1])
        dloss += bce(d3,view(-1),ytrue[:,2])
        return dloss
        
    def remove_decisions(df):
        cols = [c for c in df.columns if c not in Const.decisions ]
        ddf = df[cols]
        return ddf
    
    makeinput = lambda step,dids: df_to_torch(remove_decisions(dataset.get_input_state(step=step,ids=dids)))
    threshold = lambda x: torch.gt(x,torch.rand(x.shape[0])).type(torch.FloatTensor)

    randchoice = lambda x: x[torch.randint(len(x),(1,))[0]]
    tloss_func = torch.nn.TripletMarginLoss()
    def get_tloss(row,step,yt,x,imitation=True):
        if yt[:,step].std() < .001:
            return torch.tensor([0]).to(device)
        positive_idx= torch.nonzero(yt[:,step] == yt[row,step])
        if len(positive_idx) <= 1:
            print('no losses','n positive',len(positive_idx),'yt',yt[row,step],'row',row,'step',step,'imitation',imitation,end='\r')
            return torch.tensor([0]).to(device)
        positive_idx = torch.stack([ii for ii in positive_idx if ii != row]).view(-1)
        negative_idx = torch.tensor([ii for ii in range(x.shape[0]) if ii not in positive_idx and ii != row])
        if len(positive_idx) < 1 or len(negative_idx) < 1:
            print('no losses','n positive',len(positive_idx),'n negative',len(negative_idx),'yt',yt[row,step],'row',row,'step',step,'imitation',imitation,end='\r')
            return torch.tensor([0]).to(device)
        positive = x[randchoice(positive_idx)]
        negative = x[randchoice(negative_idx)]
        anchor = x[row]
        if use_attention:
            [anchor_embedding,pos_embedding,neg_embedding] = [model.get_embedding(xx.view(1,-1),position=step,use_saved_memory=True) for xx in [anchor,positive,negative]]
        else:    
            [anchor_embedding,pos_embedding,neg_embedding] = [model.get_embedding(xx.view(1,-1),position=step,concatenate=False)[int(imitation)] for xx in [anchor,positive,negative]]
        tloss = tloss_func(anchor_embedding,pos_embedding,neg_embedding)
        return tloss
    #save the inputs from the whole dataset for future reference
    if use_attention:
        full_data = []
        for mstep in [0,1,2]:
            full_data_step = [baseline, get_dlt(min(mstep,1)),
                         get_dlt(mstep),get_pd(mstep),get_nd(mstep),get_cc(mstep),get_mod(mstep)]
            full_data_step = torch.cat([formatdf(fd,true_ids) for fd in full_data_step],axis=1)
            full_data.append(full_data_step)
        full_data = torch.stack(full_data)
        model.save_memory(full_data)
        print(full_data.shape)
        
    def step(train=True):
        if train:
            model.train(True)
            tmodel1.train(True)
            tmodel2.train(True)
            tmodel3.train(True)
            optimizer.zero_grad()
            ids = train_ids
            y_opt = makegrad(optimal_train)
            transition_dict = {k: torch.clone(v).detach() for k,v in transitions_train.items()}
        else:
            ids = test_ids
            model.eval()
            tmodel1.eval()
            tmodel2.eval()
            tmodel3.eval()
            y_opt = makegrad(optimal_test)
            print(y_opt.mean(axis=0))
            transition_dict = {k: torch.clone(v).detach() for k,v in transitions_test.items()}
        model.set_device(device)
        ytrain = df_to_torch(outcomedf.loc[ids]).to(device)
        #imitation losses and decision 1
        xxtrained = [baseline, get_dlt(0),get_dlt(0),get_pd(0),get_nd(0),get_cc(0),get_mod(0)]
        xxtrain = [formatdf(xx,ids) for xx in xxtrained]
        xxtrain = torch.cat(xxtrain,axis=1).to(device)
        o1 = model(xxtrain,position=0,use_saved_memory= (not train))
        decision1_imitation = o1[:,3]
        decision1_opt = o1[:,0]
    
        imitation_loss1 = bce(decision1_imitation,ytrain[:,0])
        imitation_loss1 = torch.mul(imitation_loss1,imitation_weights[0])
        opt_loss1 = bce(decision1_opt,y_opt[:,0])
        opt_loss1 = torch.mul(opt_loss1,opt_weights[0])
        
        x1_imitation = [baseline, get_dlt(1),get_dlt(0),get_pd(1),get_nd(1),get_cc(1),get_mod(1)]
        x1_imitation = [formatdf(xx1,ids) for xx1 in x1_imitation]
        x1_imitation = torch.cat(x1_imitation,axis=1).to(device)
        
        o2 = model(x1_imitation,position=1,use_saved_memory= (not train))
            
        decision2_imitation = o2[:,4]
            
        imitation_loss2 =  bce(decision2_imitation,ytrain[:,1])
        imitation_loss2 = torch.mul(imitation_loss2,imitation_weights[1])
        
        
        x2_imitation = [baseline, get_dlt(1),get_dlt(2),get_pd(2),get_nd(2),get_cc(2),get_mod(2)]
        
        x2_imitation = [formatdf(xx2,ids) for xx2 in x2_imitation]
        x2_imitation = torch.cat(x2_imitation,axis=1).to(device)
        
        
        o3 = model(x2_imitation,position=2,use_saved_memory= (not train))
        
        decision3_imitation = o3[:,5]
        
        imitation_loss3 = bce(decision3_imitation,ytrain[:,2])
        imitation_loss3 = torch.mul(imitation_loss3,imitation_weights[2])
        
        opt_input2 = [
            formatdf(baseline,ids), 
            transition_dict['dlt1'],
            formatdf(get_dlt(0),ids),
            transition_dict['pd1'],
            transition_dict['nd1'], 
            formatdf(get_cc(0),ids),
            transition_dict['mod']
                 ]
        opt_input2 = [o.to(device) for o in opt_input2]

        opt_input2 = torch.cat(opt_input2,axis=1).to(device)
        decision2_opt = model(opt_input2,position=1,use_saved_memory= (not train))[:,1]
        
        opt_loss2 = bce(decision2_opt,y_opt[:,1])
        opt_loss2 = torch.mul(opt_loss2,opt_weights[1])
        
        opt_input3 = [
            formatdf(baseline,ids),
            transition_dict['dlt1'],
            transition_dict['dlt2'],
            transition_dict['pd2'],
            transition_dict['nd2'],
            transition_dict['cc'],
            transition_dict['mod'],
        ]
        opt_input3 = [o.to(device) for o in opt_input3]
        opt_input3 = torch.cat(opt_input3,axis=1).to(device)
        decision3_opt = model(opt_input3,position=2,use_saved_memory= (not train))[:,2]
        
        opt_loss3 = bce(decision3_opt,y_opt[:,2])
        opt_loss3 = torch.mul(opt_loss3,opt_weights[2])
        
        iloss = torch.add(torch.add(imitation_loss1,imitation_loss2),imitation_loss3)
        iloss = torch.mul(iloss,imitation_weight)
        
        reward_loss = torch.add(torch.add(opt_loss1,opt_loss2),opt_loss3)
        reward_loss =torch.mul(reward_loss,reward_weight)
        
        loss = torch.add(iloss,reward_loss)
        
        imitation_tloss = torch.FloatTensor([0]).to(device)
        opt_tloss = torch.FloatTensor([0]).to(device)
        n_rows = xxtrain.shape[0]
        if reward_triplet_weight + imitation_triplet_weight > 0.0001:
            for i in range(n_rows):
                
                if imitation_triplet_weight > .0001:
                    imitation_tloss += get_tloss(i,0,ytrain,xxtrain,True)
                    imitation_tloss += get_tloss(i,1,ytrain,x1_imitation,True)
                    imitation_tloss += get_tloss(i,2,ytrain,x2_imitation,True)
                if reward_triplet_weight > .0001:
                    opt_tloss += get_tloss(i,0,y_opt,xxtrain,False)
                    opt_tloss += get_tloss(i,1,y_opt,opt_input2,False)
                    opt_tloss += get_tloss(i,2,y_opt,opt_input3,False)
            loss += torch.mul(imitation_tloss[0],imitation_triplet_weight/n_rows)
            loss += torch.mul(opt_tloss[0],reward_triplet_weight/n_rows)
        losses = [iloss,reward_loss,imitation_tloss*imitation_triplet_weight/n_rows,opt_tloss*reward_triplet_weight/n_rows]
        
        if train:
            loss.backward()
            optimizer.step()
            return losses
        else:
            scores = []
            distributions = [decision1_opt.mean().item(),decision2_opt.mean().item(),decision3_opt.mean().item()]
            imitation = [decision1_imitation,decision2_imitation,decision3_imitation]
            optimal = [decision1_opt,decision2_opt,decision3_opt]
            for i,decision_im in enumerate(imitation):
                deci = decision_im.cpu().detach().numpy()
                deci0 = (deci > .5).astype(int)
                iout = ytrain[:,i].cpu().detach().numpy()
                acci = accuracy_score(iout,deci0)
                try:
                    auci = roc_auc_score(iout,deci)
                except:
                    auci = -1
                
                deco = optimal[i].cpu().detach().numpy()
                deci0 = (deco > .5).astype(int)
                oout = y_opt[:,i].cpu().detach().numpy()
                acco = accuracy_score(oout,deci0)
                try:
                    auco = roc_auc_score(oout,deco)
                except:
                    auco=-1
                scores.append({'decision': i,'optimal_auc': auco,'imitation_auc': auci,'optimal_acc': acco,'imitation_acc': acci})
            return losses, scores, distributions
        
    best_val_loss = torch.tensor(1000000000.0)
    steps_since_improvement = 0
    best_val_score = {}
    
    for epoch in range(epochs):
        losses = step(True)
        val_losses,val_metrics,val_distributions = step(False)
        vl = val_losses[0] + val_losses[1]
        for vm in val_metrics:
            vl += (-((vm['optimal_auc']*reward_weight) + (vm['imitation_auc']*imitation_weight)))/10
        if verbose:
            print('______epoch',str(epoch),'_____')
            print('val reward',val_losses[1].item())
            print('imitation reward', val_losses[0].item())
            print('distance losses',val_losses[2].item(),val_losses[-1].item())
            print('distributions',val_distributions)
            print(val_metrics)
        if vl < best_val_loss:
            best_val_loss = vl
            best_val_score = val_metrics
            best_val_distributions = val_distributions
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
        if steps_since_improvement > patience:
            break
    print('++++++++++Final+++++++++++')
    print('best',best_val_loss)
    print(best_val_score)
    model.load_state_dict(torch.load(save_file))
    model.eval()
    return model, best_val_score, best_val_loss, best_val_distributions

from Models import *
args = {
    'hidden_layers': [1000], 
    'opt_layer_size': 20, 
    'imitation_layer_size': 20, 
    'dropout': 0.25, 
    'input_dropout': 0.1, 
    'shufflecol_chance': 0.5
}

#1.8424
decision_model, decision_score, decision_loss, _ = train_decision_model_triplet(
    model1,model2,model3,smodel3,
    lr=.01,
    imitation_weight=1,
    reward_weight=1,
    patience=10,
    imitation_triplet_weight=0,
    reward_triplet_weight =0,
    verbose=True,
    weights=[1,1,1,1], #realtive weight of survival, feeding tube, aspiration, andl lrc
    tweights=[0,0,3,0], #weight for OS, LRC, FDM, and event (any + FT or AS at 6m) as time to event in weeks
    use_attention=True,
    **args)
decision_model

tensor([205.,  44.,   7.]) 389
tensor([69., 18.,  6.]) 147
torch.Size([3, 536, 85])
tensor([0.4694, 0.1224, 0.0408], grad_fn=<MeanBackward1>)
______epoch 0 _____
val reward 1.5337661504745483
imitation reward 2.2648766040802
distance losses 0.0 0.0
distributions [0.7510436177253723, 0.004403852391988039, 0.0016084234230220318]
[{'decision': 0, 'optimal_auc': 0.9234485321441843, 'imitation_auc': 0.5333969465648856, 'optimal_acc': 0.5714285714285714, 'imitation_acc': 0.891156462585034}, {'decision': 1, 'optimal_auc': 0.954349698535745, 'imitation_auc': 0.647554347826087, 'optimal_acc': 0.8775510204081632, 'imitation_acc': 0.782312925170068}, {'decision': 2, 'optimal_auc': 0.8026004728132388, 'imitation_auc': 0.6993006993006993, 'optimal_acc': 0.9591836734693877, 'imitation_acc': 0.8231292517006803}]
tensor([0.4694, 0.1224, 0.0408], grad_fn=<MeanBackward1>)
______epoch 1 _____
val reward 1.6847261190414429
imitation reward 1.781301498413086
distance losses 0.0 0.0
distributions [0.1099290

tensor([0.4694, 0.1224, 0.0408], grad_fn=<MeanBackward1>)
______epoch 12 _____
val reward 0.49431464076042175
imitation reward 1.1419498920440674
distance losses 0.0 0.0
distributions [0.5348871946334839, 0.12724439799785614, 0.0265685822814703]
[{'decision': 0, 'optimal_auc': 0.9516908212560387, 'imitation_auc': 0.623091603053435, 'optimal_acc': 0.8435374149659864, 'imitation_acc': 0.8435374149659864}, {'decision': 1, 'optimal_auc': 0.9827734711455641, 'imitation_auc': 0.6551630434782608, 'optimal_acc': 0.9523809523809523, 'imitation_acc': 0.7551020408163265}, {'decision': 2, 'optimal_auc': 0.9976359338061466, 'imitation_auc': 0.8070565797838525, 'optimal_acc': 0.9591836734693877, 'imitation_acc': 0.8163265306122449}]
tensor([0.4694, 0.1224, 0.0408], grad_fn=<MeanBackward1>)
______epoch 13 _____
val reward 0.4872055649757385
imitation reward 1.1538927555084229
distance losses 0.0 0.0
distributions [0.5186882019042969, 0.16160738468170166, 0.029097622260451317]
[{'decision': 0, 'optima

______epoch 24 _____
val reward 0.4863758981227875
imitation reward 1.4479124546051025
distance losses 0.0 0.0
distributions [0.4986669719219208, 0.13891984522342682, 0.03281077370047569]
[{'decision': 0, 'optimal_auc': 0.9581939799331104, 'imitation_auc': 0.625, 'optimal_acc': 0.9183673469387755, 'imitation_acc': 0.8503401360544217}, {'decision': 1, 'optimal_auc': 0.9788975021533161, 'imitation_auc': 0.6475543478260869, 'optimal_acc': 0.9659863945578231, 'imitation_acc': 0.7551020408163265}, {'decision': 2, 'optimal_auc': 0.9940898345153664, 'imitation_auc': 0.7453909726636999, 'optimal_acc': 0.9795918367346939, 'imitation_acc': 0.8163265306122449}]
tensor([0.4694, 0.1224, 0.0408], grad_fn=<MeanBackward1>)
______epoch 25 _____
val reward 0.5377295613288879
imitation reward 1.5792698860168457
distance losses 0.0 0.0
distributions [0.5153266787528992, 0.1599303036928177, 0.03194335848093033]
[{'decision': 0, 'optimal_auc': 0.9570791527313267, 'imitation_auc': 0.6164122137404581, 'optima

DecisionAttentionModel(
  (input_dropout): Dropout(p=0.1, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=100, out_features=1000, bias=True)
  )
  (batchnorm): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (relu): Softplus(beta=1, threshold=20)
  (sigmoid): Sigmoid()
  (softmax): Softmax(dim=1)
  (final_opt_layer): Linear(in_features=1000, out_features=100, bias=True)
  (final_imitation_layer): Linear(in_features=1000, out_features=100, bias=True)
  (final_layer): Linear(in_features=1000, out_features=6, bias=True)
  (resize_layer): Linear(in_features=89, out_features=100, bias=True)
  (attentions): ModuleList(
    (0): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
    )
  )
  (norms): ModuleList(
    (0): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
  )
  (activation): ReLU()
)

In [91]:
decision_model.set_device('cpu')
torch.save(decision_model,'../resources/decision_model.pt')
pd.DataFrame(decision_score).to_csv('../results/policy_model_score.csv')
pd.DataFrame(decision_score)

,decision,optimal_auc,imitation_auc,optimal_acc,imitation_acc
0,0,0.960981,0.614027,0.870748,0.836735
1,1,0.982343,0.624728,0.959184,0.761905
2,2,0.996454,0.797521,0.979592,0.816327


In [29]:
pd.DataFrame(decision_score)

,decision,optimal_auc,imitation_auc,optimal_acc,imitation_acc
0,0,0.958904,0.617366,0.993197,0.884354
1,1,0.865385,0.680163,0.959184,0.768707
2,2,0.800699,0.816910,0.965986,0.809524


In [ ]:
#1.8424
decision_model2, decision_score2, decision_loss2, _ = train_decision_model_triplet(
    model1,model2,model3,smodel3,
    lr=.01,
    imitation_weight=1,
    reward_weight=1,
    patience=10,
    imitation_triplet_weight=0,
    reward_triplet_weight =0,
    verbose=True,
    weights=[0,0,0,0], #realtive weight of survival, feeding tube, aspiration, andl lrc
    tweights=[1,1,0,0], #weight for OS, LRC, FDM, and event (any + FT or AS at 6m) as time to event in weeks
    use_attention=True,
    **args)

In [ ]:
def train_decision_model(
    tmodel1,
    tmodel2,
    tmodel3,
    smodel3,
    use_default_split=True,
    use_bagging_split=False,
    lr=.0001,
    epochs=10000,
    patience=50,
    weights=[0,.5,.5,0], #realtive weight of survival, feeding tube, aspiration, andl lrc
    tweights=[1,1,1,0]
    imitation_weights=[.5,1,1],#weights of imitation decisions, because ic overtrains too quickly
    imitation_weight=0.1,
    shufflecol_chance = 0.1,
    reward_weight=1,
    imitation_triplet_weight=0,
    reward_triplet_weight = 0,
    split=.7,
    resample_training=False,
    save_path='../data/models/',
    file_suffix='',
    use_gpu=True,
    use_attention=True,
    verbose=True,
    threshold_decisions=True,#convert decisiosn to binary in simulation, usually breaks it
    use_smote=False,
    validate_with_memory=True,
    **model_kwargs):
    #outdated method of doing stuff, haven't updated with new loss functions idk
    tmodel1.eval()
    tmodel2.eval()
    tmodel3.eval()
    smodel3.eval()
    train_ids, test_ids = get_tt_split(use_default_split=use_default_split,use_bagging_split=use_bagging_split,resample_training=resample_training)
    true_ids = train_ids + test_ids #for saving memory without upsampling
    if use_smote:
        dataset = DTDataset(use_smote=True,smote_ids = train_ids)
        train_ids = [i for i in dataset.processed_df.index.values if i not in test_ids]
    else:
        dataset = DTDataset()
    data = dataset.processed_df.copy()
    
    def get_dlt(state):
        if state == 2:
            return data[Const.dlt2].copy()
        d = data[Const.dlt1].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_pd(state):
        if state == 2:
            return data[Const.primary_disease_states2].copy()
        d = data[Const.primary_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_nd(state):
        if state == 2:
            return data[Const.nodal_disease_states2].copy()
        d = data[Const.nodal_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_cc(state):
        res = data[Const.ccs].copy()
        if state == 1:
            res.values[:,:] = np.zeros(res.values.shape)
        return res
    
    def get_mod(state):
        res = data[Const.modifications].copy()
        #this should have an ic condition but we don't use it anumore anywa
        return res
        
    outcomedf = data[Const.outcomes]
    baseline = dataset.get_state('baseline')
    
    def formatdf(d,dids=train_ids):
        d = df_to_torch(d.loc[dids]).to(model.get_device())
        return d
    
    def makegrad(v):
        if not v.requires_grad:
            v.requires_grad=True
        return v
    
    if use_attention:
        model = DecisionAttentionModel(baseline.shape[1],**model_kwargs)
    else:
        model_kwargs = {k:v for k,v in model_kwargs.items() if 'attention' not in k and 'embed' not in k}
        model = DecisionModel(baseline.shape[1],**model_kwargs)

    device = 'cpu'
    if use_gpu and torch.cuda.is_available():
        device = 'cuda'
        
    model.set_device(device)

    tmodel1.set_device(device)
    tmodel2.set_device(device)
    tmodel3.set_device(device)
    smodel3.set_device(device)
    
    hashcode = str(hash(','.join([str(i) for i in train_ids])))
    
    save_file = save_path + 'model_' + model.identifier +'_hash' + hashcode + file_suffix + '.tar'
    model.fit_normalizer(df_to_torch(baseline.loc[train_ids]).to(model.get_device()))
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    
    mse = torch.nn.MSELoss()
    bce = torch.nn.BCELoss()
    device = model.get_device()
    def compare_decisions(d1,d2,d3,ids):
#         ypred = np.concatenate([dd.cpu().detach().numpy().reshape(-1,1) for dd in [d1,d2,d3]],axis=1)
        ytrue = df_to_torch(outcomedf.loc[ids])
        dloss = bce(d1.view(-1),ytrue[:,0])
        dloss += bce(d2.view(-1),ytrue[:,1])
        dloss += bce(d3,view(-1),ytrue[:,2])
        return dloss
        
    def remove_decisions(df):
        cols = [c for c in df.columns if c not in Const.decisions ]
        ddf = df[cols]
        return ddf
    
    makeinput = lambda step,dids: df_to_torch(remove_decisions(dataset.get_input_state(step=step,ids=dids))).to(device)
    thresh = lambda x: torch.sigmoid(100000000*(x - .5))

    optimal_train,transitions_train = calc_optimal_decisions(dataset,train_ids,tmodel1,tmodel2,tmodel3,smodel3,
                                           weights=weights,tweights=tweights,
                                          )
    optimal_test,transitions_test = calc_optimal_decisions(dataset,test_ids,tmodel1,tmodel2,tmodel3,smodel3,
                                          weights=weights,tweights=tweights,
                                         )
    optimal_train = optimal_train.to(model.get_device())
    optimal_test = optimal_test.to(model.get_device())
    
    #save the inputs from the whole dataset for future reference
    if use_attention:
        full_data = []
        for mstep in [0,1,2]:
            full_data_step = [baseline, get_dlt(min(mstep,1)),
                         get_dlt(mstep),get_pd(mstep),get_nd(mstep),get_cc(mstep),get_mod(mstep)]
            full_data_step = torch.cat([formatdf(fd,true_ids) for fd in full_data_step],axis=1)
            full_data.append(full_data_step)
        full_data = torch.stack(full_data)
        model.save_memory(full_data.to(device))
        print(full_data.shape)
        
    randchoice = lambda x: x[torch.randint(len(x),(1,))[0]]
    tloss_func = torch.nn.TripletMarginLoss()
    def get_tloss(row,step,yt,x,imitation=True):
        if yt[:,step].std() < .001:
            return torch.tensor([0]).to(device)
        positive_idx= torch.nonzero(yt[:,step] == yt[row,step])
        positive_idx = torch.stack([ii for ii in positive_idx if ii != row]).view(-1)
        negative_idx = torch.tensor([ii for ii in range(x.shape[0]) if ii not in positive_idx and ii != row])
        if len(positive_idx) < 1 or len(negative_idx) < 1:
            print('no losses','n positive',len(positive_idx),'n negative',len(negative_idx),'yt',yt[row,step],'row',row,'step',step,'imitation',imitation,end='\r')
            return torch.tensor([0]).to(device)
        positive = x[randchoice(positive_idx)]
        negative = x[randchoice(negative_idx)]
        anchor = x[row]
        if use_attention:
            [anchor_embedding,pos_embedding,neg_embedding] = [model.get_embedding(xx.view(1,-1),position=step,use_saved_memory=True) for xx in [anchor,positive,negative]]
        else:    
            [anchor_embedding,pos_embedding,neg_embedding] = [model.get_embedding(xx.view(1,-1),position=step,concatenate=False)[int(imitation)] for xx in [anchor,positive,negative]]
        tloss = tloss_func(anchor_embedding,pos_embedding,neg_embedding)
        return tloss
    
    def step(train=True):
        if train:
            model.train(True)
            tmodel1.train(True)
            tmodel2.train(True)
            tmodel3.train(True)
            optimizer.zero_grad()
            ids = train_ids
            y_opt = makegrad(optimal_train)
        else:
            ids = test_ids
            model.eval()
            tmodel1.eval()
            tmodel2.eval()
            tmodel3.eval()
            y_opt = makegrad(optimal_test)
            
            
        ytrain = df_to_torch(outcomedf.loc[ids]).to(device)
        #imitation losses and decision 1
        xxtrained = [baseline, get_dlt(0),get_dlt(0),get_pd(0),get_nd(0),get_cc(0),get_mod(0)]
        xxtrain = torch.cat([formatdf(xx,ids) for xx in xxtrained],axis=1).to(device)
        
        use_memory = (not train) and validate_with_memory

        o1 = model(xxtrain,position=0,use_saved_memory = use_memory)

        decision1_imitation = o1[:,3]
        
        decision1_opt = o1[:,0]
        if threshold_decisions:
            decision1_opt = thresh(decision1_opt)

        imitation_loss1 = bce(decision1_imitation,ytrain[:,0])
        imitation_loss1 = torch.mul(imitation_loss1,imitation_weights[0])
        
        x1_imitation = [baseline, get_dlt(1),get_dlt(0),get_pd(1),get_nd(1),get_cc(1),get_mod(1)]
        x1_imitation = [formatdf(xx1,ids) for xx1 in x1_imitation]
        x1_imitation = torch.cat(x1_imitation,axis=1).to(device)
        decision2_imitation = model(x1_imitation,position=1,use_saved_memory = use_memory)[:,4]

        imitation_loss2 =  bce(decision2_imitation,ytrain[:,1])
        imitation_loss2 = torch.mul(imitation_loss2,imitation_weights[1])
        
        x2_imitation = [baseline, get_dlt(1),get_dlt(2),get_pd(2),get_nd(2),get_cc(2),get_mod(2)]
        
        x2_imitation = [formatdf(xx2,ids) for xx2 in x2_imitation]
        x2_imitation = torch.cat(x2_imitation,axis=1)
        decision3_imitation = model(x2_imitation,position=2,use_saved_memory = use_memory)[:,5]

        imitation_loss3 = bce(decision3_imitation,ytrain[:,2])
        imitation_loss3 = torch.mul(imitation_loss3,imitation_weights[2])
        
        #reward decisions
        xx1 = makeinput(1,ids)
        xx2 = makeinput(2,ids)
        xx3 = makeinput(3,ids)

        xx1 = makegrad(xx1)
        xx2 = makegrad(xx2)
        xx3 = makegrad(xx3)
        baseline_train_base = formatdf(baseline,ids)
            
        baseline_train = torch.clone(baseline_train_base)

        
        xi1 = torch.cat([xx1,decision1_opt.view(-1,1)],axis=1)
        print(train,tmodel1.training,tmodel1.dropout.training)
        [ypd1, ynd1, ymod, ydlt1] = tmodel1(xi1)['predictions']
        print(train,tmodel1.training,tmodel1.dropout.training)
        d1_thresh = torch.gt(decision1_opt.view(-1,1),.5).to(ypd1.device)
        d1_scale = torch.cat([d1_thresh,d1_thresh,torch.ones(d1_thresh.view(-1,1).shape).to(ypd1.device)],dim=1)
        ypd1= torch.mul(ypd1,d1_scale)
        ynd1= torch.mul(ynd1,d1_scale)
        
        x1 = [baseline_train,ydlt1,formatdf(get_dlt(0),ids),ypd1,ynd1,formatdf(get_cc(1),ids),ymod]
        x1= torch.cat([xx1.to(model.get_device()) for xx1 in x1],axis=1)
        
        decision2_opt = model(x1,position=1,use_saved_memory = use_memory)[:,1] 
        if threshold_decisions:
            decision2_opt = thresh(decision2_opt)
            
        xi2 = torch.cat([xx2,decision1_opt.view(-1,1),decision2_opt.view(-1,1)],axis=1)
        [ypd2,ynd2,ycc,ydlt2] = tmodel2(xi2)['predictions']

        x2 = [baseline_train,ydlt1,ydlt2,ypd2,ynd2,ycc,ymod]
        x2 = torch.cat([xx2.to(model.get_device()) for xx2 in x2],axis=1)
        decision3_opt = model(x2,position=2,use_saved_memory = use_memory)[:,2]
        
        if threshold_decisions:
            decision3_opt = thresh(decision3_opt)
            
        xi3 = torch.cat([xx3,decision1_opt.view(-1,1),decision2_opt.view(-1,1),decision3_opt.view(-1,1)],axis=1)
        
        outcomes = tmodel3(xi3)['predictions']
        survival = smodel3.time_to_event(xi3,n_samples=1)
        if not train and verbose:
            print(torch.mean(outcomes,dim=0))
            
        reward_loss = torch.mean(outcome_loss(outcomes,weights) + temporal_loss(survival,tweights))
        loss = torch.add(imitation_loss1,imitation_loss2)
        loss = torch.add(loss,imitation_loss3)
        loss = torch.mul(loss,imitation_weight/3)
        loss = torch.add(loss,torch.mul(reward_loss,reward_weight))
        
        imitation_tloss = torch.FloatTensor([0]).to(device)
        opt_tloss = torch.FloatTensor([0]).to(device)
        n_rows = x1.shape[0]
        if reward_triplet_weight + imitation_triplet_weight > 0.0001:
            for i in range(n_rows):
                #skip if we're using an attention model idk
                if not use_attention and imitation_triplet_weight > .0001:
                    imitation_tloss += get_tloss(i,0,ytrain,xxtrain,True)
                    imitation_tloss += get_tloss(i,1,ytrain,x1_imitation,True)
                    imitation_tloss += get_tloss(i,2,ytrain,x2_imitation,True)
                if reward_triplet_weight > .0001:
                    opt_tloss += get_tloss(i,0,y_opt,xxtrain,False)
                    opt_tloss += get_tloss(i,1,y_opt,x1,False)
                    opt_tloss += get_tloss(i,2,y_opt,x2,False)
            loss += torch.mul(imitation_tloss[0],imitation_triplet_weight/n_rows)
            loss += torch.mul(opt_tloss[0],reward_triplet_weight/n_rows)
        
        losses = [imitation_loss1+imitation_loss2+imitation_loss3,reward_loss,imitation_tloss,opt_tloss]

        if train:
            loss.backward()
            optimizer.step()
            return losses
        else:
            scores = []
            distributions = [decision1_opt.mean().item(),decision2_opt.mean().item(),decision3_opt.mean().item()]
            imitation = [decision1_imitation,decision2_imitation,decision3_imitation]
            optimal = [decision1_opt,decision2_opt,decision3_opt]
            for i,decision_im in enumerate(imitation):
                deci = decision_im.cpu().detach().numpy()
                deci0 = (deci > .5).astype(int)
                iout = ytrain[:,i].cpu().detach().numpy()
                acci = accuracy_score(iout,deci0)
                try:
                    auci = roc_auc_score(iout,deci)
                except:
                    auci = -1
                
                deco = optimal[i].cpu().detach().numpy()
                deci0 = (deco > .5).astype(int)
                oout = y_opt[:,i].cpu().detach().numpy()
                acco = accuracy_score(oout,deci0)
                try:
                    auco = roc_auc_score(oout,deco)
                except:
                    auco=-1
                scores.append({'decision': i,'optimal_auc': auco,'imitation_auc': auci,'optimal_acc': acco,'imitation_acc': acci})
            return losses, scores, distributions
        
    best_val_loss = torch.tensor(1000000000.0)
    steps_since_improvement = 0
    best_val_score = {}
    
    for epoch in range(epochs):
        losses = step(True)
        val_losses,val_metrics,val_distributions = step(False)
        vl = val_losses[0] + val_losses[1]
        for vm in val_metrics:
            vl += (-((vm['optimal_auc']*reward_weight) + (vm['imitation_auc']*imitation_weight)))/10
        if verbose:
            print('______epoch',str(epoch),'_____')
            print('val reward',val_losses[1].item())
            print('imitation reward', val_losses[0].item())
            if len(val_losses) > 2:
                print('distance losses',val_losses[2].item(),val_losses[-1].item())
            print('distributions',val_distributions)
            print(val_metrics)
        if vl < best_val_loss:
            best_val_loss = vl
            best_val_score = val_metrics
            best_val_distributions = val_distributions
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
        if steps_since_improvement > patience:
            break
    print('++++++++++Final+++++++++++')
    print('best',best_val_loss)
    print(best_val_score)
    model.load_state_dict(torch.load(save_file))
    model.eval()
    return model, best_val_score, best_val_loss, best_val_distributions

from Models import *
# args = {
#     'hidden_layers': [50,50], 
#     'attention_heads': [2,2],
#     'embed_size': 120, 
#     'dropout': 0.5, 
#     'input_dropout': 0.2, 
#     'shufflecol_chance':  0.2,
# }
args = {
    'hidden_layers': [500], 
    'opt_layer_size': 20, 
    'imitation_layer_size': 20, 
    'dropout': 0.25, 
    'input_dropout': 0.25, 
    'shufflecol_chance': 0.5
}
from Models import *
decision_model, _, _, _ = train_decision_model(
    model1,model2,model3,smodel3,
    lr=.001,
    use_attention=True,
    imitation_weight=1,
    imitation_triplet_weight=0,
    reward_triplet_weight=0,
    reward_weight=2,
    validate_with_memory=True,
    use_smote=False,
    **args)